In [1]:
from scipy import optimize

In [ ]:
thetas = np.zeros(3)
m = y.size

def sigmoid(z):
    return(1 / (1 + np.exp(-z)))

def calculate_cost(thetas, x, y):
    h = sigmoid(x.dot(thetas.reshape(-1, 1)))
    J =  (sum((y)*-1*np.log(h)+(1-y)*-1*np.log(1-h)) / m)
    
    if np.isnan(J[0]):
        return(np.inf)
    return(J[0])

def gradient(thetas, x, y):
    h = sigmoid(x.dot(thetas.reshape(-1, 1)))
    grad = (1/m)*x.T.dot(h-y)
    return(grad.flatten())

In [ ]:
minimize(
    fun=calculate_cost,
    x0=thetas,
    args=(x,y),
    method=None,
    jac=gradient,
    options={'maxiter':400}
)